# inverted_indexes_cloab

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Installing, importing, and initializing PySpark


The following code installs PySpark and its dependencies in Colab. In addition, we install GraphFrames, which is a PySpark package for dealing with graphs in a distributed fashion. Colab notebooks run on a single machine so we will work in local mode, i.e. there is no cluster of machines and both the master and worker processes run on a single machine. This will help us debug and iron out the code we will use in the second half of this assignment on an actual cluster mode in GCP. 

The installation in the next cell should take about 1 minute in a fresh environment. Don't worry about going  over the 90 seconds limit for the assignment because in our testing enviroment all of these requirements will already be met. 

In [2]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

openjdk-8-jdk-headless is already the newest version (8u352-ga-1~18.04).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
--2022-12-28 12:05:57--  https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar
Resolving repos.spark-packages.org (repos.spark-packages.org)... 18.165.83.55, 18.165.83.27, 18.165.83.25, ...
Connecting to repos.spark-packages.org (repos.spark-packages.org)|18.165.83.55|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘/usr/local/lib/python3.7/dist-packages/pyspark/jars/graphframes-0.8.2-spark3.2-s_2.12.jar’ not modified on server. Omitting download.



In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [5]:
spark

## Copy some wiki data

As mentioned above, all wiki dumps were preprocessed and placed in a shared google storage bucket. To access the files in Colab, you will first have to authenticate with google storage, and then copy the data to your local environment. 

**IMPORTANT NOTE**

Before you start working with the wiki data, you first have to go over the 
"Working with GCP.pdf" Provided to you under the gcp folder in the same zip file as this folder. In that pdf you will redeem your credits for the GCP and create your instance. You have to do this procedure because the files will be mounted to you through GCP process. 
Notice that you have to change the project_id variable in the second cell below. 

In GCP, the storage will already be mounted on the cluster machines and we will show you how to access it.


In [6]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except: 
  pass

In [7]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'dependable-data-370310'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
mkdir: cannot create directory ‘wikidumps’: File exists
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
| [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


# Processing wikipedia

Now that we completed the setup and have some data in our local environment, we are ready to process it using PySpark. 

## A 2-minute intro to PySpark

One of the fundamental data structures in spark is a **Resillient Distributed Dataset (RDD)**. It is an immutable distributed collections of objects that are partitioned across machines, and thus making parallel processing natural and easy. RDD's support two types of operations: *transformations* (e.g. selecting a subset of fields from each element or projecting each element from A->B), and *actions* (e.g. save to disk or `collect` elements into the master node's memory). Watch out for `collect` operations that send too much data to the master node, it will crash without a trace because of running out of memory. 

Spark employs **lazy-evaluation** (like many of us...), meaning that it accumulates things we ask it to do into an execution plan, but doesn't do any calculation until a result is actually needed. When a result is needed? when we save something to disk, ask spark to show the data, or use other actions. But make no mistakes, Spark is not a slacker. It employs lazy-evaluation because it allows optimizations that wouldn't be possible otherwise and it often saves users from unnecessary data reads, tranfers across machines, and other costly operations. As we will see in a bit, the RDD data structure lends itself to simple and efficient MapReduce operations.

Let's look at our data before transforming it to RDD.

In [11]:
from pathlib import Path 
import os
from collections import defaultdict

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

# לשנות את ההגבלה ל4000
parquetFile = spark.read.parquet(path)
doc_text_pairs = parquetFile.limit(4000).select("text", "id").rdd
doc_title_pairs = parquetFile.limit(4000).select("title", "id").rdd
doc_anchor_pairs = parquetFile.limit(4000).select("anchor_text").rdd

def map_func(doc_anchor):
  edge_set = []
  for id, anchor in doc_anchor[0]:
    if (id, anchor) not in edge_set:
      edge_set.append((id, anchor))
  return edge_set

doc_anchor_pairs_new = doc_anchor_pairs.flatMap(map_func).distinct().groupByKey().mapValues(list).map(lambda x:(" ".join(x[1]),x[0]))
parquetFile.show()

import json 
# Open the JSON file 
with open('queries_train.json') as f: 
# Load the JSON data into a Python dictionary 
  dic = json.load(f)

# Select the second column of the RDD 
names_column = doc_text_pairs.map(lambda x: x[1]).collect()

ddd=defaultdict(list)
for key in dic.keys():
  for doc_id in dic[key]:
    if(doc_id in names_column):
      ddd[key].append(str(doc_id))
print(ddd.items())



+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [12]:
# dict_items([('Most expensive city in the world', ['7780']),
#             ('India', ['2377']),
#             ('Apple computer', ['2275', '856', '2116', '1344', '2117']),
#             ('How to lose weight?', ['8460']),
#             ('how to deal with depression?', ['4531', '8389']),
#             ('Natural Language processing', ['5561']),
#             ('Dolly the sheep', ['6910']),
#             ('What is the best place to live in?', ['3138', '844', '8522', '3708', '5391', '5407', '1998'])])

In [13]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords =["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

all_stopwords = english_stopwords.union(corpus_stopwords)

def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in all_stopwords]    
    return list_of_tokens

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  return([(k,(id,v)) for k,v in Counter(tokens).items() if k not in all_stopwords and v>0])

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key=lambda tup: tup[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda x:(x[0],len(x[1])))

  NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  dict1 = postings.map(lambda token: (token2bucket_id(token[0]), token)).groupByKey()
  return dict1.map(InvertedIndex.write_a_posting_list)

# merge the posting locations into a single dict and run more tests
def merge(postings):
  super_posting_locs = defaultdict(list)
  for posting_loc in postings:
    for k, v in posting_loc.items():
      super_posting_locs[k].extend(v)
  return super_posting_locs

def calc_dl(tokens,doc_id):
  return (doc_id,len(tokens))

def calc_total_term(postings):
  return postings.mapValues(helpFunc).collectAsMap()

def helpFunc(posting):
  count=0
  for tup in posting:
    count+=tup[1]
  return count

In [14]:
min_title=0
min_body=10
min_anchor=0
!mkdir body_index title_index anchor_index

mkdir: cannot create directory ‘body_index’: File exists
mkdir: cannot create directory ‘title_index’: File exists
mkdir: cannot create directory ‘anchor_index’: File exists


In [17]:
import zipfile 
# #inverted index body
# index_body=InvertedIndex()
# word_count_bodys = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# body_dl=doc_text_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
# index_body.DL=body_dl

# posting_lists_body = word_count_bodys.groupByKey().mapValues(reduce_word_counts)
# filter_bodys = posting_lists_body.filter(lambda x: len(x[1])>min_body)
# index_body.term_total=calc_total_term(filter_bodys)
# index_body.df=calculate_df(filter_bodys).collectAsMap()

# posting_locs_body = partition_postings_and_write(filter_bodys).collect()
# posting_locs_body=merge(posting_locs_body)
# index_body.posting_locs=posting_locs_body
# index_body.write_index('body_index', 'index_body')

#inverted index title
# index_title=InvertedIndex()
# word_count_titles = doc_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# title_dl=doc_title_pairs.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
# index_title.DL=title_dl

# posting_lists_title = word_count_titles.groupByKey().mapValues(reduce_word_counts)
# filter_titles = posting_lists_title.filter(lambda x: len(x[1])>min_title)
# index_title.term_total=calc_total_term(filter_titles)
# index_title.df=calculate_df(filter_titles).collectAsMap()

# posting_locs_title = partition_postings_and_write(filter_titles).collect()
# posting_locs_title=merge(posting_locs_title)
# index_title.posting_locs=posting_locs_title
# index_title.write_index('title_index', 'index_title')

# # #inverted index anchor
index_anchor=InvertedIndex()
word_count_anchors = doc_anchor_pairs_new.flatMap(lambda x: word_count(x[0], x[1]))
index_dl=doc_anchor_pairs_new.map(lambda x: calc_dl(tokenize(x[0]),x[1])).collectAsMap()
index_anchor.DL=index_dl

posting_lists_anchor = word_count_anchors.groupByKey().mapValues(reduce_word_counts)
filter_anchors = posting_lists_anchor.filter(lambda x: len(x[1])>min_anchor)
index_anchor.term_total=calc_total_term(filter_anchors)
index_anchor.df=calculate_df(filter_anchors).collectAsMap()

posting_locs_anchor = partition_postings_and_write(filter_anchors).collect()
posting_locs_anchor=merge(posting_locs_anchor)
index_anchor.posting_locs=posting_locs_anchor
index_anchor.write_index('anchor_index', 'index_anchor')

# Open the zip file in write mode 
with zipfile.ZipFile('invertedAnchor.zip', 'w') as zip: 
# Add the bin file to the zip file
  for i in range(0,124):
    string=str(i)+'_000.bin'
    try: 
      zip.write(string)
    except:
      continue
